# Variáveis aleatórias:

$X_1 = R = \text{Lognormal com }\mu_R = 249,89 \text{kN e  }\sigma_R = 24.978 \text{kN}$

$X_2 = S_{pp} = \text{Normal com }\mu_{pp} = 19.86 \text{kN e  }\sigma_{pp} = 0.993 \text{kN}$

$X_3 = S_{sv} = \text{Gumbel Tipo I (máximo) com }\mu_{sv} = 78.84 \text{kN e  }\sigma_{pp} = 15.768 \text{kN}$

# Equação de Estado Limite

$g(\underline{X}) = X_1 - X_2 - X_3$

# Dados de entrada

In [1]:
# Vetor de médias:
M = [249.78, 19.86, 78.84]

# Vetor de desvios padrão:
D = [24.978, 0.993, 15.768]

# Precisão fixada:
delta = 0.00001

# Algoritmo

## Carrega as bibliotecas necessárias

In [2]:
import autograd.numpy as np
from autograd import grad
from scipy.stats import lognorm, norm, gumbel_r
from copy import copy

## Define as funções que serão utilizadas

In [3]:
# Função do estado limite
def calcular_gx(X):
    return X[0] - X[1] - X[2]


# Funções para obter os gradiente:
def __calcular_gradiente(X):
    return grad(calcular_gx)(X)

def calcular_gradiente_z(X, J):
    grad_x = __calcular_gradiente(X)
    return np.matmul(np.linalg.inv(J), grad_x)


# Função para calcular o módulo do vetor:
def calcular_modulo(vetor):
    return np.sqrt(sum([x**2 for x in vetor]))


# Função para calcular o coeficiente de variação:
def __calcular_cv(media, dpadrao):
    return dpadrao / media


# Função para calcular os parâmetros da Lognormal:
def calcular_parametros_lognormal(media, dpadrao):
    cv = __calcular_cv(media, dpadrao)
    params = {}
    params['epsilon'] = np.sqrt(np.log(1 + cv**2))
    params['lambda'] = np.log(media) - 1/2 * params['epsilon']**2
    return params


# Função para calcular os parâmetros da Gumbel:
def calcular_parametros_gumbel(media, dpadrao):
    cv = __calcular_cv(media, dpadrao)
    params = {}
    params['alpha'] = np.pi / np.sqrt(6 * dpadrao**2)
    params['u'] = media  - (0.577216 / params['alpha'])
    return params


# Função para calcular os parâmetros da Normal equivalente:
def calcular_parametros_normal_equiv(acumulada, densidade, media):
    inversa = norm.ppf(acumulada)
    phi = norm.pdf(inversa)
    params = {}
    params['sigma'] = phi / densidade
    params['mi'] = media - params['sigma'] * inversa
    return params

## Obtem os parâmetros da Lognormal ($X_1$) e Gumbel ($X_3$)

In [4]:
# Parâmetros da Lognormal
params_x1 = calcular_parametros_lognormal(M[0], D[0])
print('Lognormal:')
print(params_x1)

print('\n')

# Parâmetros da distribuição de Gumbel
params_x3 = calcular_parametros_gumbel(M[2], D[2])
print('Gumbel:')
print(params_x3)

Lognormal:
{'epsilon': 0.0997513451195927, 'lambda': 5.515605365008355}


Gumbel:
{'alpha': 0.08133877664649061, 'u': 71.74355690363208}


## Transforma os dados de entrada para o formato adequado

In [5]:
# Vetor de médias
M = np.array(M, dtype=float)

# Vetor de desvios padrão
D = np.array(D, dtype=float)

## Inicia o processo iterativo

In [18]:
# Inicializa as variáveis:
k = 0
beta = []
convergiu = False

while not convergiu:
    
    print('\n')
    print(f'> Iteração k = {k}')
    
    # Obtém o ponto do projeto no espaço original:
    if k == 0:
        X = copy(M)
    else:
        X = X + np.matmul(np.linalg.inv(J), np.subtract(Z_, Z))
    print(f'>>> Ponto (espaço original) X{k}: {X}')
    
    # Obtém os parâmetros da Normal Equivalente à Lognormal
    acum = lognorm.cdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
    dens = lognorm.pdf(x=X[0], s=params_x1['epsilon'], scale=np.exp(params_x1['lambda']))
    params_x1_equiv = calcular_parametros_normal_equiv(acum, dens, X[0])
    
    print(f'>>> Parâmetros da normal equivalente à Lognormal: {params_x1_equiv}')

    # Obtém os parâmetros da Normal Equivalente à Gumbel
    acum = gumbel_r.cdf(X[2], params_x3['u'], 1/params_x3['alpha'])
    dens = gumbel_r.pdf(X[2], params_x3['u'], 1/params_x3['alpha'])
    params_x3_equiv = calcular_parametros_normal_equiv(acum, dens, X[2])
    
    print(f'>>> Parâmetros da normal equivalente à Gumbel: {params_x3_equiv}')
    
    L = np.array([[1,0,0], [0,1,0], [0,0,1]], dtype=float) # Matriz L
    M_ = [params_x1_equiv['mi'], M[1], params_x3_equiv['mi']] # Vetor de médias'
    D_ = [params_x1_equiv['sigma'], D[1], params_x3_equiv['sigma']] # Vetor de desvios padrão'
    D_ = np.diag(np.array(D_, dtype=float)) # Matriz de desvios padrão
    D_inv = np.linalg.inv(D_) # Matriz de desvios padrão inversa
    J = np.matmul(L, np.linalg.inv(D_)) # Matriz Jacobiana
    
    print(f">>> Vetor M': {M_}")
    print(f'>>> Matriz J:')
    print(J)
    
    # Obtém o ponto do projeto no espaço reduzido:
    Z = np.matmul(J, np.subtract(X, M_))
    
    print(f'>>> Ponto (espaço reduzido) Z{k}: {Z}')
    
    # Calcula o gradiente:
    grad_z = calcular_gradiente_z(X, J)
    
    # Calcula o índice de confiabilidade:
    beta.append(calcular_modulo(Z))
    
    print(f'>>> Índice de confiabilidade (beta): {beta[k]}')
    
    # Calculo da estimativa no novo ponto
    modulo_gradz = calcular_modulo(grad_z)
    Z_ = (1 / modulo_gradz**2) *(np.matmul(grad_z, Z) - calcular_gx(X)) * grad_z
    
    
    # Verifica o critério de convergência:
    if k > 0:
        convergiu = abs(beta[k] - beta[k-1]) / beta[k] <= delta
        print(f'>>> Atendeu o critério de convergência? {convergiu}')
        
        if convergiu:
            print('\n')
            print(f'> Fim do processo iterativo')
            
            # Calcula a confiabilidade
            conf = norm.cdf(beta[k])
            pfalha = 1 - conf
            print('\n')
            print(f'Confiabilidade: {conf:.15f}')
            print(f'Probabilidade de falha: {pfalha:.15f}')
        else:
            k += 1
    else:
        k += 1



> Iteração k = 0
>>> Ponto (espaço original) X0: [249.78  19.86  78.84]
>>> Parâmetros da normal equivalente à Lognormal: {'sigma': 24.915890983971863, 'mi': 248.5373031797478}
>>> Parâmetros da normal equivalente à Gumbel: {'sigma': 15.076630546314302, 'mi': 76.16643412482047}
>>> Vetor M': [248.5373031797478, 19.86, 76.16643412482047]
>>> Matriz J:
[[0.04013503 0.         0.        ]
 [0.         1.00704935 0.        ]
 [0.         0.         0.06632782]]
>>> Ponto (espaço reduzido) Z0: [0.04987567 0.         0.17733179]
>>> Índice de confiabilidade (beta): 0.18421223167506437


> Iteração k = 1
>>> Ponto (espaço original) X1: [137.03119584  20.0371105  116.99408534]
>>> Parâmetros da normal equivalente à Lognormal: {'sigma': 13.669046108434724, 'mi': 218.61912562120125}
>>> Parâmetros da normal equivalente à Gumbel: {'sigma': 29.127182870653773, 'mi': 59.85204676166721}
>>> Vetor M': [218.61912562120125, 19.86, 59.85204676166721]
>>> Matriz J:
[[0.07315799 0.         0.        ]
 